## Preparación de audio:

Voy a generar un diccionrio key = persona y value = lista de audios, para a continuación unir todos los audios de cada persona. El objetivo es tener un único audio por persona que luego trocearé para entrenar el Autoencoder.

In [1]:
import os

In [2]:
people = os.listdir("../inputs/LibriSpeech/test-other")
print(people)

['6070', '7105', '533', '3764', '8188', '3331', '3997', '2414', '4852', '.DS_Store', '6938', '6128', '8280', '3528', '5442', '4198', '4350', '7902', '367', '3538', '8461', '4294', '3080', '7018', '5764', '1998', '3005', '2609', '7975', '2033', '6432', '8131', '5484', '1688']


In [3]:
people.remove(".DS_Store") 

In [4]:
all_files = {}
for p in people:
    list_dir = os.listdir(f'../inputs/LibriSpeech/test-other/{p}')
    if '.DS_Store' in list_dir:
        list_dir.remove(".DS_Store")
    file_dict ={}
    for l in list_dir:
        files = [x for x in os.listdir(f'../inputs/LibriSpeech/test-other/{p}/{l}') if x.endswith(".flac")]
        file_dict[l]= files
    
    all_files[p] = file_dict



In [5]:
all_files['6070']

{'86744': ['6070-86744-0027.flac',
  '6070-86744-0011.flac',
  '6070-86744-0007.flac',
  '6070-86744-0006.flac',
  '6070-86744-0010.flac',
  '6070-86744-0026.flac',
  '6070-86744-0017.flac',
  '6070-86744-0001.flac',
  '6070-86744-0021.flac',
  '6070-86744-0020.flac',
  '6070-86744-0000.flac',
  '6070-86744-0016.flac',
  '6070-86744-0003.flac',
  '6070-86744-0015.flac',
  '6070-86744-0019.flac',
  '6070-86744-0023.flac',
  '6070-86744-0022.flac',
  '6070-86744-0018.flac',
  '6070-86744-0014.flac',
  '6070-86744-0002.flac',
  '6070-86744-0009.flac',
  '6070-86744-0025.flac',
  '6070-86744-0005.flac',
  '6070-86744-0013.flac',
  '6070-86744-0029.flac',
  '6070-86744-0028.flac',
  '6070-86744-0012.flac',
  '6070-86744-0004.flac',
  '6070-86744-0024.flac',
  '6070-86744-0008.flac'],
 '63485': ['6070-63485-0007.flac',
  '6070-63485-0011.flac',
  '6070-63485-0010.flac',
  '6070-63485-0006.flac',
  '6070-63485-0001.flac',
  '6070-63485-0017.flac',
  '6070-63485-0016.flac',
  '6070-63485-0000.

In [6]:
#Unifico audios por persona y cambio a formato MP3:

from pydub import AudioSegment

for p_id, p_info in all_files.items():
    #print(p_id)
    #print(p_info)
    audio = 0
    for key, values in p_info.items():
        #print (key)
        #print(values)
        for v in values:
            audio += AudioSegment.from_file(f'../inputs/LibriSpeech/test-other/{p_id}/{key}/{v}', format = 'flac')
    
    audio.export(f'../outputs/combined_sound/{p_id}.mp3', format='mp3')
    


Troceo los audios en audios de 20', con un overlap de 50%. Antes obtendré para cada persona el tiempo total de audio, y descartaré aquellos que no cumplan un mínimo.

In [7]:
milis=6000000000
time = []
for p in people:
    origin = AudioSegment.from_file(f'../outputs/combined_sound/{p}.mp3', format='mp3')
    time.append((p,len(origin)))
    if len(origin)<milis:
        milis=len(origin)
    

In [8]:
time

[('6070', 600650),
 ('7105', 609135),
 ('533', 602895),
 ('3764', 607120),
 ('8188', 607045),
 ('3331', 604705),
 ('3997', 618325),
 ('2414', 605840),
 ('4852', 612240),
 ('6938', 158355),
 ('6128', 607685),
 ('8280', 489100),
 ('3528', 615785),
 ('5442', 606750),
 ('4198', 608110),
 ('4350', 607431),
 ('7902', 605190),
 ('367', 404205),
 ('3538', 601990),
 ('8461', 607810),
 ('4294', 610370),
 ('3080', 607685),
 ('7018', 614280),
 ('5764', 603485),
 ('1998', 600955),
 ('3005', 602230),
 ('2609', 605865),
 ('7975', 602615),
 ('2033', 601605),
 ('6432', 601425),
 ('8131', 602670),
 ('5484', 606425),
 ('1688', 589595)]

In [9]:
#Descarto para el análisis  los audios con time<600000 para que el resultado final quede balanceado 
# (número de audios por persona similar): 

discarted = [e[0] for e in time if e[1]<600000]
discarted

['6938', '8280', '367', '1688']

In [10]:
#Aplico un overlap 50%:

interval = 20000

for p in people:
    if p not in discarted:
        origin = AudioSegment.from_file(f'../outputs/combined_sound/{p}.mp3', format='mp3')
        num = (len(origin)//interval)*2
        counter = 0

        for i in range(num):
            if counter == 0:
                splited = origin[0:interval]
                counter+=int(interval/2)
                interval+=int(counter)
            else: 
                splited = origin[counter:interval]
                counter+=10000
                interval+=10000

            splited.export(f'../outputs/splited_audio/{p}_{i}.mp3', format='mp3')

            #print(f'Exportado {p}_{i}.mp3 - audio_time= {len(splited)}')

    #print('FIN!')